In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from pprint import pprint

In [2]:
load_dotenv()

True

In [3]:
from dataclasses import dataclass

@dataclass
class LanguageContext:
    user_language: str = "English"

@dynamic_prompt
def user_language_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role. """

    user_language = request.runtime.context.user_language
    base_prompt = "You are a helpful assistant."

    if user_language != "English":
        return f"{base_prompt} only respond in {user_language}."
    elif user_language == "English":
        return base_prompt

In [4]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")

In [6]:
agent = create_agent(
    model=model,
    context_schema=LanguageContext,
    middleware=[user_language_prompt]
)

In [13]:
response = agent.invoke(
    {
        "messages": [HumanMessage(content="Hello, how are you?")]
    },
    context=LanguageContext(user_language="Irish")
)


In [14]:
print(response["messages"][-1].content)

Dia duit! Tá mé go maith, go raibh maith agat. Conas atá tú?


In [18]:
response = agent.invoke(
    {"messages": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Spanish")
)

In [19]:
print(response["messages"][-1].content)

Hola, estoy bien, ¿y tú?


In [20]:
response = agent.invoke(
    {"messages": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="French")
)

print(response["messages"][-1].content)

Bonjour, comment allez-vous ?
